In [ ]:
# Install required packages
!pip install transformers accelerate bitsandbytes torch sentencepiece sqlparse
!pip uninstall bitsandbytes -y
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Step 1: Authenticate with your Hugging Face token
# Replace 'YOUR_HF_TOKEN' with the token you copied
login(token="your-HF_TOKEN")

# Model setup - Using Gemma 7B Instruct
model_id = "google/gemma-7b-it"

# Load tokenizer with auth
tokenizer = AutoTokenizer.from_pretrained(model_id)

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,  # Use this instead of load_in_4bit
    torch_dtype=torch.float16
)
# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

In [ ]:
DBMS_KNOWLEDGE = {
    "acid": """ACID stands for:

- **Atomicity**: Ensures all operations in a transaction are completed. If not, the transaction is aborted.
- **Consistency**: Maintains the integrity of the database before and after the transaction.
- **Isolation**: Ensures that transactions do not interfere with each other.
- **Durability**: Once a transaction is committed, the changes are permanent.

**Example:**
```sql
BEGIN TRANSACTION;
  UPDATE accounts SET balance = balance - 500 WHERE id = 1;
  UPDATE accounts SET balance = balance + 500 WHERE id = 2;
COMMIT;
If anything fails, the transaction rolls back and no money is lost.
"""
}


DBMS_PROMPT_TEMPLATE = """<start_of_turn>system
You are DBExpert, a highly advanced database management system specialist with expertise in:
SQL (MySQL, PostgreSQL, SQL Server, Oracle)
NoSQL (MongoDB, Cassandra, Redis)
PL/SQL, T-SQL
Database design (normalization, ER modeling)
Query optimization
Transactions and concurrency control
Distributed databases
Data warehousing concepts

You provide:
Accurate, executable queries for any requested operation
Detailed explanations when asked "why" or "how"
Theoretical foundations when requested
Performance analysis and optimization suggestions
Comparisons between different database technologies

Format responses as:
Code blocks for queries
Bullet points for multiple items
Clear section headers for complex answers
Unless asked for brevity, provide comprehensive responses with examples where helpful.<end_of_turn>
<start_of_turn>user
{user_query}<end_of_turn>
<start_of_turn>assistant
"""

def sanitize_response(raw):
    raw = html.unescape(raw)
    raw = re.sub(r'<[^>]*>', '', raw) # Remove HTML tags
    raw = re.sub(r'[^ -~\n\t]+', '', raw) # Remove non-ASCII junk
    raw = re.sub(r'\s+', ' ', raw) # Normalize whitespace
    raw = raw.strip()
    return raw

# Response generation
@lru_cache(maxsize=100)
def generate_response(user_input, chat_history=()):
    context = ""
    for i, (q, a) in enumerate(chat_history[-3:]):
        context += f"<start_of_turn>user\nPrevious Question {i+1}: {q}<end_of_turn>\n"
        context += f"<start_of_turn>assistant\nPrevious Answer {i+1}: {a}<end_of_turn>\n"

        full_prompt = f"{context}{DBMS_PROMPT_TEMPLATE.format(user_query=user_input)}"

        outputs = pipe(full_prompt)
        response = outputs[0]["generated_text"].split("<start_of_turn>assistant")[-1]
        response = response.replace("<end_of_turn>", "").strip()
        response = sanitize_response(response)

    for concept in DBMS_KNOWLEDGE:
        if concept in user_input.lower():
        response += f"\n\n🧠 *Extra Knowledge — {concept.upper()}*\n{DBMS_KNOWLEDGE[concept]}"

    return response

In [ ]:
# Chat interface loop
def chat_interface():
    history = []
    print("DBExpert: Hello! I'm your DBMS expert assistant. How can I help you today?")
    print("Type 'exit' or 'quit' to end the session.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit"]:
            print("\nDBExpert: Thank you for using DBExpert. Goodbye!")
            break

        response = generate_response(user_input, tuple(history))
        print(f"\nDBExpert: {response}\n")
        history.append((user_input, response))

# Run chatbot
if __name__ == "__main__":
    chat_interface()

# **ANOTHER IMPLEMENTATION**

In [ ]:
!pip install transformers sentence-transformers faiss-cpu


  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.2 MB/s eta 0:00:00
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Use a small free instruction-tuned model for reasonable response time
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")



tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import json
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

# Load DBMS knowledge base
kb_json = {
  "SQL DDL": "DDL commands include CREATE, ALTER, DROP, and TRUNCATE. They define the structure of a database.",
  "SQL DML": "DML commands include SELECT, INSERT, UPDATE, DELETE. They manipulate data in tables.",
  "SQL Joins": "Joins are used to retrieve data from multiple tables. INNER JOIN returns matching rows. LEFT JOIN returns all from left table and matched from right. FULL JOIN combines both.",
  "SQL Functions": "SQL has aggregate functions like COUNT, SUM, AVG, MIN, MAX and scalar functions like UCASE, LCASE, MID, LEN.",
  "SQL Subqueries": "Subqueries are nested queries within another query using SELECT, FROM, or WHERE clauses.",
  "SQL Constraints": "Constraints enforce data integrity: PRIMARY KEY, FOREIGN KEY, UNIQUE, NOT NULL, CHECK.",
  "SQL Index": "Indexes speed up search queries on columns. UNIQUE index enforces uniqueness.",
  "SQL Views": "A View is a virtual table created by a query. Can be used to simplify complex queries.",
  "MongoDB CRUD": "MongoDB uses db.collection.insertOne(), find(), updateOne(), deleteOne() for CRUD operations.",
  "MongoDB Aggregation": "Aggregation processes data records and returns computed results using $match, $group, $project.",
  "MongoDB Indexing": "Indexes improve performance. Created via db.collection.createIndex({ field: 1 })",
  "MongoDB Schema Design": "MongoDB is schema-less. Embedded documents and referencing are used based on access patterns.",
  "PLSQL Loops": "PL/SQL supports FOR loops, WHILE loops, and basic LOOP...END LOOP.",
  "PLSQL Cursors": "Explicit cursors in PL/SQL allow row-by-row processing of query results using OPEN, FETCH, CLOSE.",
  "PLSQL Procedures": "Procedures are named PL/SQL blocks stored in the database. Created using CREATE PROCEDURE.",
  "PLSQL Triggers": "Triggers are procedures that execute in response to DML events like INSERT, UPDATE, DELETE.",
  "NoSQL Definition": "NoSQL databases are non-relational and store data as documents, key-value pairs, wide-columns, or graphs.",
  "NoSQL Types": "Key-Value (Redis), Document (MongoDB), Column (Cassandra), Graph (Neo4j).",
  "CAP Theorem": "CAP states a distributed database can guarantee only two: Consistency, Availability, Partition Tolerance.",
  "DBMS Normalization": "Normalization reduces redundancy. 1NF removes repeating groups, 2NF removes partial dependencies, 3NF removes transitive dependencies.",
  "Transactions": "A transaction is a unit of work with ACID properties: Atomicity, Consistency, Isolation, Durability.",
  "Concurrency Control": "Techniques include locking, timestamp ordering to handle simultaneous transactions.",
  "Indexing": "Indexes improve query speed by allowing fast lookup. B-tree and Hash indexing are common types.",
  "ER Model": "Entity-Relationship model uses entities, attributes, and relationships to design databases. Represented using ER diagrams.",
  "Relational Algebra": "A procedural query language that uses operations like select (σ), project (π), join (⨝), and union (∪).",
  "Functional Dependency": "A relationship that exists when one attribute uniquely determines another. Basis for normalization.",
  "ACID Properties": "Atomicity ensures all-or-nothing, Consistency ensures valid state, Isolation prevents concurrent conflicts, Durability ensures permanence.",
  "Deadlock": "Deadlock is a situation where two or more transactions wait forever for each other to release locks.",
  "2PL Protocol": "Two-Phase Locking ensures serializability by dividing the locking process into growing and shrinking phases.",
  "BCNF": "Boyce-Codd Normal Form is a stricter version of 3NF where every determinant is a candidate key.",
  "Denormalization": "Denormalization adds redundancy for performance optimization in some use cases.",
  "OLTP vs OLAP": "OLTP supports real-time transaction processing; OLAP supports data analysis and complex queries.",
  "Data Warehousing": "A data warehouse is a system used for reporting and data analysis, often with OLAP tools."
}

# Embed sentences
embedder = SentenceTransformer("all-MiniLM-L6-v2")
kb_keys = list(kb_json.keys())
kb_vals = list(kb_json.values())
embeddings = embedder.encode(kb_vals, convert_to_numpy=True)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def retrieve_context(question, top_k=2):
    q_emb = embedder.encode([question])[0]
    D, I = index.search(np.array([q_emb]), top_k)
    return [kb_vals[i] for i in I[0]]

def generate_answer(prompt, max_tokens=1024):
    # Fix: assign pad_token
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


    return tokenizer.decode(output[0], skip_special_tokens=True)

def build_prompt(user_input):
    context = retrieve_context(user_input)
    prompt = (
        "You are a DBMS expert assistant.\n"
        "Answer clearly and helpfully.\n\n"
        "Please explain this in detail with full example code. Do not stop early."
        "Include SQL code examples where relevant.\n\n"
        f"Context:\n{context[0]}\n{context[1]}\n\n"
        f"User: {user_input}\nAssistant:"
    )
    return prompt



In [ ]:
print("DBMS Chatbot (Offline Model). Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Bye 👋")
        break1
    prompt = build_prompt(user_input)
    answer = generate_answer(prompt)
    print(f"Bot: {answer.split('Assistant:')[-1].strip()}\n")


DBMS Chatbot (Offline Model). Type 'exit' to quit.

You: tell me about mysql


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Bot: Here are some examples of SQL code:

* SELECT:
```
SELECT * FROM customers;
```
This will return all the data from the "customers" table.

* INSERT:
```
INSERT INTO customers (name, email) VALUES ('John Doe', 'johndoe@example.com');
```
This will add a new row to the "customers" table with the name "John Doe" and email address "johndoe@example.com".

* UPDATE:
```
UPDATE customers SET email = 'johndoe@example.com' WHERE name = 'John Doe';
```
This will update the email address of the customer with the name "John Doe" to "johndoe@example.com".

* DELETE:
```
DELETE FROM customers WHERE name = 'John Doe';
```
This will remove the row from the "customers" table with the name "John Doe".

* JOIN:
```
SELECT customers.name, orders.order_date
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id;
```
This will return the name of the customer and the order date for each order, joining the "customers" and "orders" tables

